In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
print(tf.__version__)
print(sns.__version__)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)

2.15.0
0.13.1
1.25.2
2.0.3
1.2.2


##**1. 빈도수 단어 - 1000, 3000, 5000 적용해보기**

In [9]:
from tensorflow.keras.datasets import reuters

freq = [1000,3000,5000]

In [13]:
import numpy as np
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences

freq = [1000, 3000, 5000]

for i in freq:
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=i, test_split=0.2)

    # 단어 인덱스를 필터링하는 작업이 필요 없음. 이미 num_words에 따라 필터링됨.

    print(f'훈련 샘플의 수_{i}: {len(x_train)}')
    print(f'테스트 샘플의 수_{i}: {len(x_test)}')
    print(f'훈련용 뉴스의 최대 길이_{i}: {max(len(l) for l in x_train)}')
    print(f'훈련용 뉴스의 평균 길이_{i}: {sum(map(len, x_train)) / len(x_train)}')

    unique_elements, counts_elements = np.unique(y_train, return_counts=True)
    print(f"각 클래스 빈도수_{i}:")
    print(np.asarray((unique_elements, counts_elements)))



훈련 샘플의 수_1000: 8982
테스트 샘플의 수_1000: 2246
훈련용 뉴스의 최대 길이_1000: 2376
훈련용 뉴스의 평균 길이_1000: 145.5398574927633
각 클래스 빈도수_1000:
[[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
    14   15   16   17   18   19   20   21   22   23   24   25   26   27
    28   29   30   31   32   33   34   35   36   37   38   39   40   41
    42   43   44   45]
 [  55  432   74 3159 1949   17   48   16  139  101  124  390   49  172
    26   20  444   39   66  549  269  100   15   41   62   92   24   15
    48   19   45   39   32   11   50   10   49   19   19   24   36   30
    13   21   12   18]]
훈련 샘플의 수_3000: 8982
테스트 샘플의 수_3000: 2246
훈련용 뉴스의 최대 길이_3000: 2376
훈련용 뉴스의 평균 길이_3000: 145.5398574927633
각 클래스 빈도수_3000:
[[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
    14   15   16   17   18   19   20   21   22   23   24   25   26   27
    28   29   30   31   32   33   34   35   36   37   38   39   40   41
    42   43   44   45]
 [  55  432   74 3159 1949   17   48   16 

=> 이 뉴스 데이터는 3번, 4번 클래스가 대부분을 차지하고 있습니다. 그 뒤로는 19번,16번, 1번, 11번 등이 높은 분포를 가지고 있네요.

In [23]:
word_index = reuters.get_word_index()


In [25]:
import numpy as np
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Reuters 데이터셋 로드
word_index = reuters.get_word_index()
freq = [1000, 3000, 5000]

# word_index의 키-값을 뒤집어 역매핑하기 쉽게 만듭니다.
reverse_word_index = {value: key for key, value in word_index.items()}

for i in freq:
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=i, test_split=0.2)

    decoded_train = []
    decoded_test = []

    # 훈련 데이터 디코딩
    for seq in x_train:
        decoded_seq = ' '.join([reverse_word_index.get(index - 3, '?') for index in seq])
        if decoded_seq.strip():  # 빈 문장은 제외
            decoded_train.append(decoded_seq)

    # 테스트 데이터 디코딩
    for seq in x_test:
        decoded_seq = ' '.join([reverse_word_index.get(index - 3, '?') for index in seq])
        if decoded_seq.strip():  # 빈 문장은 제외
            decoded_test.append(decoded_seq)

    if not decoded_train or not decoded_test:
        print(f"Warning: No valid documents found for num_words={i}")
        continue

    x_train_i = decoded_train
    x_test_i = decoded_test

    # DTM 생성하기
    dtmvector = CountVectorizer(stop_words=None)
    x_train_dtm_i = dtmvector.fit_transform(x_train_i)
    x_test_dtm_i = dtmvector.transform(x_test_i)

    # TF-IDF 생성하기
    tfidf_transformer = TfidfTransformer()
    x_train_tfidf_i = tfidf_transformer.fit_transform(x_train_dtm_i)
    x_test_tfidf_i = tfidf_transformer.transform(x_test_dtm_i)

    # 나이브 베이즈 분류기
    model_NB = MultinomialNB().fit(x_train_tfidf_i, y_train)

    # 예측하기
    predicted_i = model_NB.predict(x_test_tfidf_i)
    print(f"정확도_나이브_{i}: {accuracy_score(y_test, predicted_i)}")
    print(classification_report(y_test, predicted_i, zero_division=0))

    # 랜덤포레스트
    model_RF = RandomForestClassifier(random_state=0).fit(x_train_tfidf_i, y_train)
    predicted_i = model_RF.predict(x_test_tfidf_i)
    print(f"정확도_랜덤포레스트_{i}: {accuracy_score(y_test, predicted_i)}")
    print(classification_report(y_test, predicted_i, zero_division=0))

    # 그래디언트 부스팅 트리
    model_GB = GradientBoostingClassifier(random_state=0).fit(x_train_tfidf_i, y_train)
    predicted_i = model_GB.predict(x_test_tfidf_i)
    print(f"정확도_그래디언트부스팅트리_{i}: {accuracy_score(y_test, predicted_i)}")
    print(classification_report(y_test, predicted_i, zero_division=0))


정확도_나이브_1000: 0.7079252003561888
              precision    recall  f1-score   support

           0       0.80      0.33      0.47        12
           1       0.45      0.80      0.58       105
           2       0.00      0.00      0.00        20
           3       0.93      0.87      0.90       813
           4       0.67      0.93      0.78       474
           5       0.00      0.00      0.00         5
           6       1.00      0.07      0.13        14
           7       0.00      0.00      0.00         3
           8       0.68      0.39      0.50        38
           9       0.92      0.48      0.63        25
          10       0.88      0.23      0.37        30
          11       0.43      0.83      0.57        83
          12       0.00      0.00      0.00        13
          13       0.61      0.38      0.47        37
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         9
          16       0.53      0.75      0.62     

어? 와이라누???? 그래디언트 부스팅 트리 5000 파라미터 변경해보기

In [27]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)
decoded_train = []
decoded_test = []

# 훈련 데이터 디코딩
for seq in x_train:
    decoded_seq = ' '.join([reverse_word_index.get(index - 3, '?') for index in seq])
    if decoded_seq.strip():  # 빈 문장은 제외
        decoded_train.append(decoded_seq)

# 테스트 데이터 디코딩
for seq in x_test:
    decoded_seq = ' '.join([reverse_word_index.get(index - 3, '?') for index in seq])
    if decoded_seq.strip():  # 빈 문장은 제외
        decoded_test.append(decoded_seq)

x_train = decoded_train
x_test = decoded_test

# DTM 생성하기
dtmvector = CountVectorizer(stop_words=None)
x_train_dtm = dtmvector.fit_transform(x_train)
x_test_dtm = dtmvector.transform(x_test)

# TF-IDF 생성하기
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_dtm)
x_test_tfidf = tfidf_transformer.transform(x_test_dtm)

# 그래디언트 부스팅 트리
gbc = GradientBoostingClassifier(random_state=0, max_depth =1).fit(x_train_tfidf, y_train)#원래 기본값 3인데 1로 줄임 (과적합 방지)
predicted = gbc.predict(x_test_tfidf)
print(f"정확도_그래디언트부스팅트리_{i}: {accuracy_score(y_test, predicted)}")
print(classification_report(y_test, predicted, zero_division=0))



정확도_그래디언트부스팅트리_5000: 0.784060552092609
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.85      0.70      0.76       105
           2       0.67      0.70      0.68        20
           3       0.93      0.91      0.92       813
           4       0.70      0.88      0.78       474
           5       0.00      0.00      0.00         5
           6       0.93      0.93      0.93        14
           7       1.00      0.33      0.50         3
           8       0.66      0.71      0.68        38
           9       0.92      0.88      0.90        25
          10       0.84      0.87      0.85        30
          11       0.66      0.67      0.67        83
          12       1.00      0.46      0.63        13
          13       0.61      0.51      0.56        37
          14       0.00      0.00      0.00         2
          15       0.31      0.56      0.40         9
          16       0.72      0.78      0.7

=> 0.78 나온 그래디언트 부스팅 5000 선택. 아마 이전거는..... 심각하게 과적합된 게 아닐까?

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, Conv1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
vocab_size = 5000
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=vocab_size, test_split=0.2)

In [40]:
#훈련용 데이터셋 길이 분포
train_lengths = [len(seq) for seq in X_train]
percentiles = np.percentile(train_lengths, [50, 75, 90, 95, 99])
mean_length = np.mean(train_lengths)
std_length = np.std(train_lengths)
mean_2sd_range = (mean_length - 2 * std_length, mean_length + 2 * std_length)

print(f"\nnum_words={i}에 대한 통계 정보:")
print(f"50% 길이: {percentiles[0]}")
print(f"75% 길이: {percentiles[1]}")
print(f"90% 길이: {percentiles[2]}")
print(f"95% 길이: {percentiles[3]}")
print(f"99% 길이: {percentiles[4]}")
print(f"평균 길이: {mean_length}")
print(f"2SD 범위: {mean_2sd_range}")


num_words=5000에 대한 통계 정보:
50% 길이: 95.0
75% 길이: 179.0
90% 길이: 313.0
95% 길이: 459.0
99% 길이: 718.0
평균 길이: 145.5398574927633
2SD 범위: (-146.08840277542225, 437.16811776094886)


In [41]:
max_len = 300
X_train_pad = pad_sequences(X_train, maxlen=max_len)
X_test_pad = pad_sequences(X_test, maxlen=max_len)

In [43]:
print('X_train의 크기(shape) :',X_train_pad.shape)
print('X_test의 크기(shape) :',X_test_pad.shape)

X_train의 크기(shape) : (8982, 300)
X_test의 크기(shape) : (2246, 300)


In [29]:
embedding_dim = 256
dropout_ratio = 0.2
num_filters = 256  #커널 수
kernel_size = 3
hidden_units = 128 #뉴런 수

In [44]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=embedding_dim), #vocab_size, embedding dim
    Dropout(dropout_ratio),
    Conv1D(num_filters, kernel_size, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(hidden_units, activation='relu'),
    Dropout(dropout_ratio),
    Dense(46, activation='softmax')
])

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         1280000   
                                                                 
 dropout_2 (Dropout)         (None, None, 256)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 256)         196864    
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                      

In [46]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [47]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data =(X_test_pad, y_test), callbacks=[es, model_checkpoint])

Epoch 1/10
141/141 [==============================] - 83s 574ms/step - loss: 1.9423 - accuracy: 0.5370 - val_loss: 1.3799 - val_accuracy: 0.6736
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


141/141 [==============================] - 89s 633ms/step - loss: 1.1659 - accuracy: 0.7280 - val_loss: 1.0426 - val_accuracy: 0.7538
Epoch 3/10
141/141 [==============================] - 76s 541ms/step - loss: 0.8696 - accuracy: 0.7937 - val_loss: 0.9004 - val_accuracy: 0.7854
Epoch 4/10
141/141 [==============================] - 81s 571ms/step - loss: 0.6721 - accuracy: 0.8342 - val_loss: 0.8523 - val_accuracy: 0.7983
Epoch 5/10
141/141 [==============================] - 77s 547ms/step - loss: 0.5165 - accuracy: 0.8713 - val_loss: 0.8514 - val_accuracy: 0.7956
Epoch 6/10
141/141 [==============================] - 77s 543ms/step - loss: 0.4019 - accuracy: 0.8986 - val_loss: 0.8360 - val_accuracy: 0.8054
Epoch 7/10
141/141 [==============================] - 76s 543ms/step - loss: 0.3076 - accuracy: 0.9196 - val_loss: 0.8932 - val_accuracy: 0.8063
Epoch 8/10
141/141 [==============================] - 79s 558ms/step - loss: 0.2450 - accuracy: 0.9352 - val_loss: 0.9312 - val_accuracy: 0.8

In [48]:
loaded_model = load_model('best_model.h5')
print("테스트 정확도: %.4f" % (loaded_model.evaluate(X_test_pad, y_test)[1]))

71/71 [==============================] - 5s 66ms/step - loss: 0.8360 - accuracy: 0.8054
테스트 정확도: 0.8054


## **회고-1**

- 딥러닝 1D CNN 이 가장 성능이 우수했다.
- ML 모델들 중에서는 그래디언트 부스팅 (중간에 이상하긴 했지만) 이 가장 성능이 좋았다.
- 오늘 하면서 알게 된 건데 1d cnn에서 average poolinng보다 maxpooling을 사용하는 것이 이 테스크의 목적에 부합한다. 여러 가지 특징들 중 가장 중요한 것만뽑아주는 max pooling은, 특징을 다소 희석하는 average pooling보다 텍스트 분류 작업에서 좋을 수 있다. 보통 자연어 처리같은 데이터에서 맥스 풀링이 더 나은 성능을 보인다고 한다.
- 근데 1D CNN은 accuracy는 높지만 loss도 높다. 이건 아마... 데이터셋에 클래스 불균형이 있어서 그런 걸수도 있다. 가령 이 데이터는 class 3,4만 엄청 많다. 이럴 때는 특정 클래스에 과대적합, 과소적합 될 수 있다. 그리고 손실 함수는 모든 샘플에 대해 동일한 가중치를 부여하는데, 이 때 지배적인 클래스의 손실이 전체 손실 값이 큰 영향을 미치게 된다.

In [49]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# 클래스 라벨
classes = np.unique(y_train)
# 클래스별 샘플 수
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
# 클래스 가중치를 딕셔너리로 변환
class_weight_dict = dict(zip(classes, class_weights))

In [51]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=embedding_dim), #vocab_size, embedding dim
    Dropout(dropout_ratio),
    Conv1D(num_filters, kernel_size, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(hidden_units, activation='relu'),
    Dropout(dropout_ratio),
    Dense(46, activation='softmax')
])

In [52]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data =(X_test_pad, y_test), class_weight = class_weight_dict, callbacks=[es, model_checkpoint])

Epoch 1/10
141/141 [==============================] - 81s 554ms/step - loss: 3.8007 - accuracy: 0.0953 - val_loss: 3.5725 - val_accuracy: 0.3615
Epoch 2/10
141/141 [==============================] - 92s 651ms/step - loss: 3.0834 - accuracy: 0.4292 - val_loss: 2.0435 - val_accuracy: 0.6300
Epoch 3/10
141/141 [==============================] - 80s 568ms/step - loss: 1.6584 - accuracy: 0.6348 - val_loss: 1.7045 - val_accuracy: 0.6705
Epoch 4/10
141/141 [==============================] - 85s 604ms/step - loss: 1.0427 - accuracy: 0.6973 - val_loss: 1.2763 - val_accuracy: 0.7115
Epoch 5/10
141/141 [==============================] - 77s 550ms/step - loss: 0.7133 - accuracy: 0.7308 - val_loss: 1.4101 - val_accuracy: 0.6821
Epoch 6/10
141/141 [==============================] - 77s 547ms/step - loss: 0.5149 - accuracy: 0.7690 - val_loss: 1.1228 - val_accuracy: 0.7449
Epoch 7/10
141/141 [==============================] - 86s 613ms/step - loss: 0.3975 - accuracy: 0.8073 - val_loss: 1.0373 - val_ac

##**회고-2**
아까보다 성능이 더 안좋다. 암만 가중치를 줬다고 해도 데이터 수 자체가 적어서 그런 걸까?

In [56]:
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
print(class_counts)

{0: 55, 1: 432, 2: 74, 3: 3159, 4: 1949, 5: 17, 6: 48, 7: 16, 8: 139, 9: 101, 10: 124, 11: 390, 12: 49, 13: 172, 14: 26, 15: 20, 16: 444, 17: 39, 18: 66, 19: 549, 20: 269, 21: 100, 22: 15, 23: 41, 24: 62, 25: 92, 26: 24, 27: 15, 28: 48, 29: 19, 30: 45, 31: 39, 32: 32, 33: 11, 34: 50, 35: 10, 36: 49, 37: 19, 38: 19, 39: 24, 40: 36, 41: 30, 42: 13, 43: 21, 44: 12, 45: 18}


In [57]:
unique, counts = np.unique(y_test, return_counts=True)
class_counts = dict(zip(unique, counts))
print(class_counts)

{0: 12, 1: 105, 2: 20, 3: 813, 4: 474, 5: 5, 6: 14, 7: 3, 8: 38, 9: 25, 10: 30, 11: 83, 12: 13, 13: 37, 14: 2, 15: 9, 16: 99, 17: 12, 18: 20, 19: 133, 20: 70, 21: 27, 22: 7, 23: 12, 24: 19, 25: 31, 26: 8, 27: 4, 28: 10, 29: 4, 30: 12, 31: 13, 32: 10, 33: 5, 34: 7, 35: 6, 36: 11, 37: 2, 38: 3, 39: 5, 40: 10, 41: 8, 42: 3, 43: 6, 44: 5, 45: 1}
